<a href="https://colab.research.google.com/github/Ullas33/Machine_Learning/blob/main/basic_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:

!pip install -q streamlit opencv-python-headless onnxruntime-gpu gTTS moviepy cloudflared textblob kornia basicsr realesrgan
!apt-get install -y ffmpeg psmisc

import os, requests, subprocess, time
import torch, numpy as np
import cv2

# Download Advanced Engines
def get_model(name, url):
    if not os.path.exists(name):
        print(f"📥 Fetching {name}...")
        r = requests.get(url)
        with open(name, "wb") as f: f.write(r.content)

get_model("AnimeGANv3_Hayao_36.onnx", "https://github.com/TachibanaYoshino/AnimeGANv3/raw/master/deploy/AnimeGANv3_Hayao_36.onnx")
get_model("action_theme.mp3", "https://www.soundhelix.com/examples/mp3/SoundHelix-Song-8.mp3")
get_model("calm_theme.mp3", "https://www.soundhelix.com/examples/mp3/SoundHelix-Song-2.mp3")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 21.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 121.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.4/299.4 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 129.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 25.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
psmisc is already the newest

In [26]:
%%writefile anime_saas_4k.py
import streamlit as st
import cv2
import numpy as np
import onnxruntime as ort
import subprocess
import os
import torch
from gtts import gTTS
from textblob import TextBlob
from moviepy.editor import VideoFileClip, AudioFileClip, CompositeAudioClip

# --- AGENT CORE: THE AETHER ENGINE ---
class AetherEngine:
    def __init__(self):
        # Optimization: Use CUDA FP16 for 2x speedup on T4
        opts = ort.SessionOptions()
        self.session = ort.InferenceSession("AnimeGANv3_Hayao_36.onnx",
                                            providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
        self.prev_frame = None # For Temporal Smoothing

    def temporal_smooth(self, current_frame, alpha=0.7):
        """Reduces flickering by blending with the previous frame's style."""
        if self.prev_frame is None:
            self.prev_frame = current_frame
            return current_frame
        smoothed = cv2.addWeighted(current_frame, alpha, self.prev_frame, 1-alpha, 0)
        self.prev_frame = smoothed
        return smoothed

    def transform_4k(self, frame, intensity, upscale):
        h, w = frame.shape[:2]
        # Neural Processing
        blob = cv2.resize(frame, (512, 512)).astype(np.float32) / 127.5 - 1.0
        out = self.session.run(None, {self.session.get_inputs()[0].name: np.expand_dims(blob, axis=0)})[0][0]
        out = ((out + 1.0) * 127.5).clip(0, 255).astype(np.uint8)

        # Upscaling & Blending
        target_res = (3840, 2160) if upscale else (w, h)
        anime_f = cv2.resize(out, target_res, interpolation=cv2.INTER_CUBIC)
        anime_f = self.temporal_smooth(anime_f) # Temporal Coherence Step

        orig_res = cv2.resize(frame, target_res)
        return cv2.addWeighted(orig_res, 1 - intensity, anime_f, intensity, 0)

    def produce_video(self, in_p, out_p, intensity, script, upscale):
        cap = cv2.VideoCapture(in_p)
        fps = cap.get(cv2.CAP_PROP_FPS)
        tw, th = (3840, 2160) if upscale else (int(cap.get(3)), int(cap.get(4)))

        # PRO-GRADE FFMPEG PIPE (H.264 High Profile)
        cmd = ['ffmpeg', '-y', '-f', 'rawvideo', '-vcodec', 'rawvideo', '-s', f'{tw}x{th}',
               '-pix_fmt', 'bgr24', '-r', str(fps), '-i', '-', '-c:v', 'libx264', '-preset', 'ultrafast', '-crf', '18', '-pix_fmt', 'yuv420p', 'v_temp.mp4']
        proc = subprocess.Popen(cmd, stdin=subprocess.PIPE)

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret: break
            processed = self.transform_4k(frame, intensity, upscale)
            proc.stdin.write(processed.tobytes())
        proc.stdin.close(); proc.wait(); cap.release()

        # INTELLIGENT AUDIO MAPPING
        v_clip = VideoFileClip('v_temp.mp4')
        audio_stack = [v_clip.audio.volumex(0.1) if v_clip.audio else None]

        if script.strip():
            gTTS(text=script, lang='en').save("voice.mp3")
            audio_stack.append(AudioFileClip("voice.mp3"))
            # Sentiment Analysis for Music Choice
            score = TextBlob(script).sentiment.polarity
            music = "calm_theme.mp3" if score > 0 else "action_theme.mp3"
            audio_stack.append(AudioFileClip(music).volumex(0.2).set_duration(v_clip.duration))

        final_audio = CompositeAudioClip([a for a in audio_stack if a is not None])
        v_clip.set_audio(final_audio).write_videofile(out_p, codec='libx264')

# --- AGENT UI: THE DIRECTOR'S CANVAS ---
st.set_page_config(page_title="AetherAnime Agent", layout="wide")
st.markdown("""
<style>
    @import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;700&display=swap');
    .stApp { background: #0E1117; color: #E0E0E0; font-family: 'Inter'; }
    .scene-box { border-radius: 20px; background: #161B22; border: 1px solid #30363D; padding: 25px; margin-bottom: 20px; }
    .stButton>button { background: linear-gradient(90deg, #1A73E8, #0D47A1); color: white; border-radius: 8px; border: none; width: 100%; }
</style>
""", unsafe_allow_html=True)

if 'beats' not in st.session_state: st.session_state.beats = [{"id": 0, "done": False, "file": ""}]

with st.sidebar:
    st.title("🛡️ Agent Status")
    st.success("Aether Engine: Online")
    upscale_4k = st.toggle("Unlock 4K Production", value=True)
    st.divider()
    if st.button("🎬 Final Master Export"):
        files = [b['file'] for b in st.session_state.beats if b['done']]
        if len(files) > 1:
            with open("f.txt", "w") as f:
                for fl in files: f.write(f"file '{fl}'\n")
            subprocess.run(['ffmpeg', '-y', '-f', 'concat', '-safe', '0', '-i', 'f.txt', '-c', 'copy', 'master.mp4'])
            st.video("master.mp4")
            with open("master.mp4", "rb") as f: st.download_button("Download Final 4K Movie", f, "anime_master.mp4")

st.title("⛩️ AetherAnime Agent")
st.caption("Advanced Video-to-Anime Production Suite • 2026 Edition")

for i, beat in enumerate(st.session_state.beats):
    st.markdown(f"<div class='scene-box'>", unsafe_allow_html=True)
    c1, c2 = st.columns([1, 1.2])
    with c1:
        st.write(f"### Story Beat {i+1}")
        v_file = st.file_uploader(f"Input Scene", type=['mp4', 'mov'], key=f"v{i}")
        script = st.text_area("Director's Script (AI Narrator)", key=f"t{i}")
        if v_in := v_file:
            if st.button(f"Generate Anime Beat {i+1}", key=f"b{i}"):
                with open(f"tmp_{i}.mp4", "wb") as f: f.write(v_in.getbuffer())
                agent = AetherEngine()
                out = f"beat_{i}.mp4"
                with st.status("Agent is painting frames..."):
                    agent.produce_video(f"tmp_{i}.mp4", out, 0.85, script, upscale_4k)
                st.session_state.beats[i]['done'] = True
                st.session_state.beats[i]['file'] = out
                st.rerun()
    with c2:
        if beat['done']: st.video(beat['file'])
        else: st.info("Waiting for Director's Input...")
    st.markdown("</div>", unsafe_allow_html=True)

if st.button("➕ Add Story Beat"):
    st.session_state.beats.append({"id": len(st.session_state.beats), "done": False, "file": ""})
    st.rerun()





Overwriting anime_saas_4k.py


In [27]:
!fuser -k 8501/tcp
subprocess.Popen(["streamlit", "run", "anime_saas_4k.py", "--server.port", "8501"])
time.sleep(10)
print("\n" + "═"*50)
print("🛡️ AETHER ANIME AGENT IS DEPLOYED")
print("1. Copy this IP:", end=" ")
!curl -s ipv4.icanhazip.com
print("2. Click this link and paste the IP to enter the Studio:")
!npx localtunnel --port 8501


══════════════════════════════════════════════════
🛡️ AETHER ANIME AGENT IS DEPLOYED
1. Copy this IP: 136.118.3.212
2. Click this link and paste the IP to enter the Studio:
⠙⠹⠸⠼⠴⠦⠧⠇⠏your url is: https://three-waves-thank.loca.lt
^C
